# Work in Progress

In [1]:
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [12]:
# data I/O
data = open('./data/train_corpus.txt', 'r').read() # should be simple plain text file
# data = data[:1000]
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_idx = { ch:i for i,ch in enumerate(chars) }
idx_to_char = { i:ch for i,ch in enumerate(chars) }

data has 858720 characters, 67 unique.


In [3]:
# BOOOOOO Python loops too slowwww 👎
class RNN(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super(RNN, self).__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        
        self.Wxh = nn.Parameter(torch.randn(hidden_size, vocab_size) * 1e-2) # Xavier init
        self.Whh = nn.Parameter(torch.randn(hidden_size, hidden_size) * 1e-2)
        self.Why = nn.Parameter(torch.randn(vocab_size, hidden_size) * 1e-2)
        self.bh = nn.Parameter(torch.zeros(hidden_size, 1))
        self.by = nn.Parameter(torch.zeros(vocab_size, 1))
        
    def forward(self, input, hidden):
        x = torch.zeros(vocab_size, 1)
        x[input] = 1
        hidden = torch.tanh(torch.mm(self.Wxh, x) + torch.mm(self.Whh, hidden + self.bh))
        output = torch.mm(self.Why, hidden) + self.by # unnormalized log probs
        output = torch.softmax(output, 0)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(hidden_size, 1)
    
# hyperparameters
hidden_size = 51
seq_length = 23
lr = 1e-2
n_epochs = 400

def train_slow(data, vocab_size, hidden_size, seq_length, lr, n_epochs):
    rnn = RNN(vocab_size, hidden_size)

    criterion = nn.NLLLoss()
    optimizer = optim.Adam(rnn.parameters(), lr=lr)
    batch_size = len(data) // seq_length 
    for epoch in range(100):
        loss = 0
        for batch_i in range(batch_size):
            rnn.zero_grad()
            hidden = rnn.init_hidden()
            s = data[batch_i*seq_length:(batch_i+1)*seq_length+1]
            input = [char_to_idx[c] for c in s[:-1]]
            target = torch.Tensor([char_to_idx[c] for c in 
                                   s[1:]]).unsqueeze(1).long()
            for t in range(seq_length-1):
                output, hidden = rnn(input[t], hidden)
                this_target = torch.tensor([target[t]])
                loss += criterion(torch.log(output).T, target[t])
        loss /= len(input) * num_seqs
        loss.backward()
        optimizer.step()
        if epoch%10==0: print("Epoch {}, Loss: {}".format(epoch, loss))
    return rnn

# model = train_slow(data, vocab_size, hidden_size, seq_length, lr, n_epochs)

def test(data, model):
    offset = 0
    with torch.no_grad():
        s = data[offset*seq_length:(offset+1)*seq_length+50]
        input = [char_to_idx[c] for c in s]
        print(s)
        sys.stdout.write(s[0])
        hidden = model.init_hidden()
        for i in input:
            output, hidden = model(i, hidden)
            next_char = idx_to_char[int(output.argmax())]
            sys.stdout.write(next_char)
# test(data, model)